# Train rt-detr to detect weeds in potato fields

In this notebook we'll train a [rt-detr](https://docs.ultralytics.com/models/rtdetr/) model to detect weeds in a potato fields. 
The target weeds are:
- kikuyo
- lengua de vaca
- diente de león

These weeds are common in potatoes fields. The images was captured in 9-10 meters height with a drone. Then, the images was selected and cropped 
to 1920x1920 pixels. Finally, the images was labeled with Roboflow.

## Install dependencies
First of all, we'll check the GPU, and install the dependencies.

In [1]:
!nvidia-smi
!pip install -q coco-eval pycocotools supervision ultralytics wandb

Sun May 12 15:46:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Initialize weights & biases
We'll use weights & biases to log the training process.

In [2]:
import wandb
from kaggle_secrets import UserSecretsClient

project_name = 'rt-detr-potatoes'
user_secrets = UserSecretsClient()
key = user_secrets.get_secret('wandb')


wandb.login(key=key)
wandb.init(project=project_name, job_type='training')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aluissp. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240512_154642-1msbvo00
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run apricot-leaf-1
wandb: ⭐️ View project at https://wandb.ai/aluissp/rt-detr-potatoes
wandb: 🚀 View run at https://wandb.ai/aluissp/rt-detr-potatoes/runs/1msbvo00


# Create data.yaml
We'll create a data.yaml file with the classes and the paths to the images.

In [3]:
import yaml

data_yaml = '''
path: /kaggle/input/potatoes-weeds-dataset/potatoes-dataset.v1-dataset-without-diente-leon.yolov8-obb

train: train/images
val: valid/images
test: test/images

names:
  0: diente_leon
  1: kikuyo
  2: lengua_vaca
  3: papa
'''

data = yaml.safe_load(data_yaml)

with open('data.yaml', 'w') as file:
    yaml.dump(data, file)

## Prepare model
We'll download the rt-detr model and prepare it before training.

In [4]:
from ultralytics import RTDETR
from wandb.integration.ultralytics import add_wandb_callback


# Load a COCO-pretrained RT-DETR-l model
model = RTDETR('rtdetr-l.pt')

# Display model information (optional)
model.info()

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


100%|██████████| 63.4M/63.4M [00:00<00:00, 215MB/s]


rt-detr-l summary: 673 layers, 32970476 parameters, 0 gradients, 108.3 GFLOPs


RTDETR(
  (model): DetectionModel(
    (model): Sequential(
      (0): HGStem(
        (stem1): Conv(
          (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (stem2a): Conv(
          (conv): Conv2d(32, 16, kernel_size=(2, 2), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (stem2b): Conv(
          (conv): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (stem3): Conv(
          (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, mome

## Train the model
We'll train the model.

In [5]:
results = model.train(project=project_name,data='data.yaml', epochs=50, imgsz=640, device=[0, 1], batch=8)
results = model.val()

wandb.finish()

Ultralytics YOLOv8.2.13 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=rtdetr-l.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=rt-detr-potatoes, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, 

100%|██████████| 755k/755k [00:00<00:00, 14.1MB/s]
2024-05-12 15:47:10,900	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-12 15:47:12,557	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-12 15:47:15.377855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 15:47:15.377962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 15:47:15.541915: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS facto

Overriding model.yaml nc=80 with nc=4
WARNING ⚠️ no model scale passed. Assuming scale='l'.

                   from  n    params  module                                       arguments                     
  0                  -1  1     25248  ultralytics.nn.modules.block.HGStem          [3, 32, 48]                   
  1                  -1  6    155072  ultralytics.nn.modules.block.HGBlock         [48, 48, 128, 3, 6]           
  2                  -1  1      1408  ultralytics.nn.modules.conv.DWConv           [128, 128, 3, 2, 1, False]    
  3                  -1  6    839296  ultralytics.nn.modules.block.HGBlock         [128, 96, 512, 3, 6]          
  4                  -1  1      5632  ultralytics.nn.modules.conv.DWConv           [512, 512, 3, 2, 1, False]    
  5                  -1  6   1695360  ultralytics.nn.modules.block.HGBlock         [512, 192, 1024, 5, 6, True, False]
  6                  -1  6   2055808  ultralytics.nn.modules.block.HGBlock         [1024, 192, 1024, 5, 

2024-05-12 15:47:35.178758: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 15:47:35.178811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 15:47:35.180464: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorBoard: Start with 'tensorboard --logdir rt-detr-potatoes/train', view at http://localhost:6006/


...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:47<00:00, 11.76s/it]


                   all         29       4099      0.585       0.63      0.636      0.389
                kikuyo         29        297      0.298      0.212      0.222      0.108
           lengua_vaca         29        596      0.602      0.743      0.738      0.355
                  papa         29       3206      0.854      0.936      0.947      0.704
Ultralytics YOLOv8.2.13 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
rt-detr-l summary: 494 layers, 32148140 parameters, 0 gradients, 103.8 GFLOPs


Generating Visualizations for batch-1/1:   0%|          | 0/8 [00:00<?, ?it/s]

Speed: 0.3ms preprocess, 506.0ms inference, 0.0ms loss, 88.9ms postprocess per image
Results saved to rt-detr-potatoes/train2


wandb:                                                                                
wandb: 🚀 View run apricot-leaf-1 at: https://wandb.ai/aluissp/rt-detr-potatoes/runs/1msbvo00
wandb: ⭐️ View project at: https://wandb.ai/aluissp/rt-detr-potatoes
wandb: Synced 5 W&B file(s), 1 media file(s), 1 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240512_154642-1msbvo00/logs


## Save the model
We'll save the model in the output folder.

In [6]:
model.export()

Ultralytics YOLOv8.2.13 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
rt-detr-l summary: 498 layers, 31991960 parameters, 0 gradients, 103.4 GFLOPs

PyTorch: starting from 'rt-detr-potatoes/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 300, 8) (63.1 MB)

TorchScript: starting export with torch 2.1.2...
TorchScript: export success ✅ 5.4s, saved as 'rt-detr-potatoes/train/weights/best.torchscript' (123.0 MB)

Export complete (6.2s)
Results saved to /kaggle/working/rt-detr-potatoes/train/weights
Predict:         yolo predict task=detect model=rt-detr-potatoes/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=rt-detr-potatoes/train/weights/best.torchscript imgsz=640 data=data.yaml  
Visualize:       https://netron.app


'rt-detr-potatoes/train/weights/best.torchscript'